In [35]:
import pandas as pd
import numpy as np
from borehole_tables import get_tables

docid = '35454'

In [36]:
bhtables = get_tables(docid, bh=True)

In [74]:
bhtables[0]

,Hole,Easting (m),Northing (m),R.L. (m),Completed,T.D. (m),B of Cz (m),B of W (m)
0,CHO100,557869.50,7595340.20,229.40,25/08/01,Abn 60,nd,nd
1,CHO10OC,557866.40,7595340.40,229.44,11/03/02,204.85,15.0,117.0
2,CH0101,557102.30,7595134.01,226.62,26/08/01,Abn 153,nd,nd
3,CH0102C,562411.71,7614561.13,241.44,22/07/03,252.44,142.4,160.10
4,CHO103C,575990.91,7610262.94,253.62,27/07/03,174.34,78.0,92.5


In [75]:
bhtables[1]

,Lance Grimstone & Associates Pty Ltd,DATE: Aug 2003
0,Projection: AGD84: Z55,NaN
1,Compiled by: SCALE: 1:250 000,FIGURE: 2


In [66]:
bh_col_str = 'Hole ID, borehole nu, drillhole, no. hole, hole no., Hole No. (Site No.), bore, hole, well, drill hole, hole #, bore or well reference number, bore or well reference nu, borehole, uphole, holeid, bh_id, well name, viell'
bh_key_str = 'Identifying name of the petroleum well, well name and number, well, well name, drill hole, hole number,  well name and, well no, borehole no, well number'

bh_loc_col_str = 'Survey ea, survey no, surveyed ea, surveyed no, collar (local easting, coordinates grid) northing, northing, easting, Easting AMG (Local), Northing AMG (Local), Co-ords, collar, Local E & N, Co-Ordinates North/East, collar east, collar north, AMG east, AMG north, zone, AMG Zone, East AGD66 Zone 54, North AGD66 Zone 54, AMGEASTIN, AMGNORTH, east, north, Easting (m), Northing (m) '
bh_loc_key_str = 'Latitude, longitude, northing, easting, surveyed location, AMG Zone I, lat, long, location, grid (amg), grid location, map zone / sheet, field location'

strs = [bh_col_str, bh_key_str, bh_loc_col_str, bh_loc_key_str]
arrays = []

for s in strs:
    a = s.split(',')
    a = [preprocess_str(w) for w in a]
    arrays.append(a)
    
bh_col = arrays[0]
bh_key = arrays[1]
bh_loc_col = arrays[2]
bh_loc_key = arrays[3]


In [67]:
bh_col, bh_key, bh_loc_col, bh_loc_key

(['hole id',
  'borehole nu',
  'drillhole',
  'no hole',
  'hole no',
  'hole no site no',
  'bore',
  'hole',
  'well',
  'drill hole',
  'hole',
  'bore or well reference number',
  'bore or well reference nu',
  'borehole',
  'uphole',
  'holeid',
  'bhid',
  'well name',
  'viell'],
 ['identifying name of the petroleum well',
  'well name and number',
  'well',
  'well name',
  'drill hole',
  'hole number',
  'well name and',
  'well no',
  'borehole no',
  'well number'],
 ['survey ea',
  'survey no',
  'surveyed ea',
  'surveyed no',
  'collar local easting',
  'coordinates grid northing',
  'northing',
  'easting',
  'easting amg local',
  'northing amg local',
  'coords',
  'collar',
  'local e  n',
  'coordinates northeast',
  'collar east',
  'collar north',
  'amg east',
  'amg north',
  'zone',
  'amg zone',
  'east agd66 zone 54',
  'north agd66 zone 54',
  'amgeastin',
  'amgnorth',
  'east',
  'north',
  'easting m',
  'northing m'],
 ['latitude',
  'longitude',
  'nor

In [65]:
import string 

def preprocess_str(s):
    s = s.lower()
    s = s.translate(str.maketrans('', '', string.punctuation))
    s = s.strip(' ')
    return s

print(preprocess_str('         hey sis!      '))

hey sis


## Easiest approach: Trying to find matching column name(s) 

In [172]:
def extract_from_columns(table):
    loc = 0
    bh_source = None
    loc1_source = None
    loc2_source = None
    zone_source = None
    for name in table.columns:
        proc_name = preprocess_str(name)
        if proc_name in bh_col:
            print('bh name: ', name)
            bh_source = name
        if proc_name in bh_loc_col:
            print('bh loc name: ', name)
            if loc == 0:
                loc1_source = name
                loc += 1
            elif loc == 1:
                loc2_source = name
            elif loc == 2:
                zone_source = name
            else:
                print('more than three loc? :', name)
    if bh_source:
        bhs = table[bh_source].values
        loc1s = None
        loc2s = None
        zone = None
        if loc1_source:
            loc1s = table[loc1_source].values
        if loc2_source:
            loc2s = table[loc2_source].values
        if zone_source:
            zone = table[zone].values
        assert len(bhs) == len(loc1s) == len(loc2s)
        
        rows = len(bhs)
        print(rows)
        add_to_df = [bhs, loc1s, loc2s, zone, [bh_source], [loc1_source], [loc2_source], [zone_source]]
        for i in range(len(add_to_df)):
            if add_to_df[i] is None:
                add_to_df[i] = [None for e in range(rows)]  
                #print(add_to_df[i])
            elif len(add_to_df[i]) < rows:
                if len(add_to_df[i]) == 1:
                    add_to_df[i] = [add_to_df[i][0] for e in range(rows)]
                    #print(add_to_df[i])
                    
        #print(add_to_df)
        dfdata = [pd.Series(a) for a in add_to_df]
        
        #i = 0
        #for series in dfdata:
        #    series.name = table_data_cols[i]
        #    i += 1
        #print(dfdata)
        df_dict = {key: value for key, value in zip(table_data_cols, dfdata)}
        add_to_df = pd.DataFrame(df_dict, columns=table_data_cols)
        print(add_to_df)
        return add_to_df
    return None

In [173]:
bh_data_cols = ['DocID', 'BH', 'location1', 'location2', 'zone', 
                'BH source', 'location1 source', 'location2 source', 'zone source']
table_data_cols = ['BH', 'location1', 'location2', 'zone', 
                'BH source', 'location1 source', 'location2 source', 'zone source']
bh_data = pd.DataFrame(columns=bh_data_cols)

In [174]:
for table in bhtables:
    res = extract_from_columns(table)
    #if isinstance(res, pd.DataFrame):
    bh_data = bh_data.append(res, ignore_index=True)

bh name:  Hole 
bh loc name:  Easting (m) 
bh loc name:  Northing (m) 
5
         BH  location1   location2  zone BH source location1 source  \
0   CHO100   557869.50  7595340.20  None     Hole      Easting (m)    
1  CHO10OC   557866.40  7595340.40  None     Hole      Easting (m)    
2   CH0101   557102.30  7595134.01  None     Hole      Easting (m)    
3  CH0102C   562411.71  7614561.13  None     Hole      Easting (m)    
4  CHO103C   575990.91  7610262.94  None     Hole      Easting (m)    

  location2 source zone source  
0    Northing (m)         None  
1    Northing (m)         None  
2    Northing (m)         None  
3    Northing (m)         None  
4    Northing (m)         None  


In [176]:
bh_data['DocID'] = docid
bh_data

,DocID,BH,location1,location2,zone,BH source,location1 source,location2 source,zone source
0,35454,CHO100,557869.50,7595340.20,None,Hole,Easting (m),Northing (m),None
1,35454,CHO10OC,557866.40,7595340.40,None,Hole,Easting (m),Northing (m),None
2,35454,CH0101,557102.30,7595134.01,None,Hole,Easting (m),Northing (m),None
3,35454,CH0102C,562411.71,7614561.13,None,Hole,Easting (m),Northing (m),None
4,35454,CHO103C,575990.91,7610262.94,None,Hole,Easting (m),Northing (m),None


In [178]:
bh_data.values

array([['35454', 'CHO100 ', 557869.5, 7595340.2, None, 'Hole ',
        'Easting (m) ', 'Northing (m) ', None],
       ['35454', 'CHO10OC ', 557866.4, 7595340.4, None, 'Hole ',
        'Easting (m) ', 'Northing (m) ', None],
       ['35454', 'CH0101 ', 557102.3, 7595134.01, None, 'Hole ',
        'Easting (m) ', 'Northing (m) ', None],
       ['35454', 'CH0102C ', 562411.71, 7614561.13, None, 'Hole ',
        'Easting (m) ', 'Northing (m) ', None],
       ['35454', 'CHO103C ', 575990.91, 7610262.94, None, 'Hole ',
        'Easting (m) ', 'Northing (m) ', None]], dtype=object)

In [180]:
from csv import writer
import os

def append_rows(file_name, df):
    # Open file in append mode
    write_cols = False
    if not os.path.exists(file_name):
        write_cols = True
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        if write_cols:
            csv_writer.writerow(df.columns.values)
        csv_writer.writerows(df.values)

fname = 'bh_refs.csv'
append_rows(fname, bh_data)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [27]:
t = bhtables[0].to_numpy()
t = np.insert(t, 0, bhtables[0].columns.values)
t = t.tolist()
t = [str(doc) for doc in t]

In [28]:
vectorizer = CountVectorizer(analyzer='char')
X = vectorizer.fit_transform(t)

In [32]:
sim = cosine_similarity(X)

In [34]:
for row in sim:
    print (row)

[1.         0.35856858 0.43386092 0.3721042  0.64549722 0.24806947
 0.45643546 0.4662524  0.4472136  0.         0.         0.
 0.12403473 0.2981424  0.25819889 0.25819889 0.51639778 0.
 0.         0.         0.11547005 0.16903085 0.2        0.15811388
 0.26967994 0.         0.         0.         0.12403473 0.28284271
 0.25819889 0.25819889 0.25819889 0.         0.         0.
 0.11547005 0.13483997 0.15811388 0.13483997 0.42426407 0.
 0.         0.         0.11547005 0.1490712  0.2        0.2       ]
[0.35856858 1.         0.77784447 0.51887452 0.46291005 0.59299945
 0.6000992  0.61300603 0.17817416 0.         0.         0.
 0.14824986 0.53452248 0.46291005 0.46291005 0.15430335 0.
 0.         0.         0.13801311 0.20203051 0.23904572 0.18898224
 0.16116459 0.         0.         0.         0.14824986 0.50709255
 0.46291005 0.46291005 0.15430335 0.         0.         0.
 0.13801311 0.16116459 0.18898224 0.16116459 0.16903085 0.
 0.         0.         0.13801311 0.17817416 0.23904572 0.